# Exercício 8

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
import tfgraphviz as tfg
from keras.layers import Dense
from tensorboard import notebook

In [2]:
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [4]:
#reset_graph()
tf.compat.v1.reset_default_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs") 
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
n_epochs = 400
batch_size = 50

min_value = 1.0
n_iteration = 0
loss_batch = []

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch}) 
            temp_loss = sess.run(loss, feed_dict={X: X_batch, y: y_batch})  
         
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     
        
        if(temp_loss < min_value):
            min_value = temp_loss
            n_iteration = 0
        else:
            n_iteration += 1
          
        print('Loss = ', min_value)
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
        if(n_iteration >= 50):
            break;
    
    print('Early stopping executed!')
    save_path = saver.save(sess, "./my_model_finalv4.ckpt")

Loss =  0.11989795
0 Train accuracy: 0.96 Test accuracy: 0.9592
Loss =  0.034007482
1 Train accuracy: 1.0 Test accuracy: 0.9688
Loss =  0.015141343
2 Train accuracy: 1.0 Test accuracy: 0.9726
Loss =  0.015141343
3 Train accuracy: 0.98 Test accuracy: 0.9748
Loss =  0.015141343
4 Train accuracy: 0.98 Test accuracy: 0.9762
Loss =  0.015141343
5 Train accuracy: 0.98 Test accuracy: 0.9804
Loss =  0.015141343
6 Train accuracy: 0.98 Test accuracy: 0.9796
Loss =  0.00924704
7 Train accuracy: 1.0 Test accuracy: 0.9816
Loss =  0.004659665
8 Train accuracy: 1.0 Test accuracy: 0.9816
Loss =  0.001119045
9 Train accuracy: 1.0 Test accuracy: 0.98
Loss =  0.001119045
10 Train accuracy: 1.0 Test accuracy: 0.9836
Loss =  0.001119045
11 Train accuracy: 1.0 Test accuracy: 0.9834
Loss =  0.001119045
12 Train accuracy: 1.0 Test accuracy: 0.9828
Loss =  0.00012941117
13 Train accuracy: 1.0 Test accuracy: 0.9822
Loss =  0.00012941117
14 Train accuracy: 1.0 Test accuracy: 0.9822
Loss =  0.00012941117
15 Train

Loss =  9.822713e-07
124 Train accuracy: 1.0 Test accuracy: 0.984
Loss =  9.822713e-07
125 Train accuracy: 1.0 Test accuracy: 0.9842
Loss =  9.822713e-07
126 Train accuracy: 1.0 Test accuracy: 0.984
Loss =  9.822713e-07
127 Train accuracy: 1.0 Test accuracy: 0.9842
Loss =  9.822713e-07
128 Train accuracy: 1.0 Test accuracy: 0.9848
Loss =  9.822713e-07
129 Train accuracy: 1.0 Test accuracy: 0.9838
Loss =  9.822713e-07
130 Train accuracy: 1.0 Test accuracy: 0.984
Loss =  9.822713e-07
131 Train accuracy: 1.0 Test accuracy: 0.9842
Early stopping executed!


In [8]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv4.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_finalv4.ckpt


In [9]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [10]:
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

In [12]:
%tensorboard --logdir ./

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.